In [1]:
import random, os
import numpy as np
import torch
import glob

def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [3]:
import pandas as pd

test_df = pd.read_csv('test/test.csv')
test_df['date_orig'] = pd.to_datetime(test_df['date'], format='%Y-%m-%d %H:%M:%S')
test_df.head()

,id,date,date_orig
0,135,2022-05-22 11:51:06,2022-05-22 11:51:06
1,136,2022-05-22 11:51:25,2022-05-22 11:51:25
2,137,2022-05-22 12:04:27,2022-05-22 12:04:27
3,138,2022-05-22 12:07:56,2022-05-22 12:07:56
4,139,2022-05-22 12:08:10,2022-05-22 12:08:10


In [4]:
parsed_time_test_df=pd.read_csv('parsed_time_test.txt',sep='\t',names=['frame','date'])
parsed_time_test_df['date_orig'] = pd.to_datetime(parsed_time_test_df['date'], format='%m-%d-%Y %a %H:%M:%S')
parsed_time_test_df['date'] = parsed_time_test_df['date_orig'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
# parsed_time_test_df=parsed_time_test_df.set_index('frame')
parsed_time_test_df.head()

,frame,date,date_orig
0,1,2022-05-22 08:25:46,2022-05-22 08:25:46
1,2,2022-05-22 08:25:46,2022-05-22 08:25:46
2,3,2022-05-22 08:25:47,2022-05-22 08:25:47
3,4,2022-05-22 08:25:47,2022-05-22 08:25:47
4,5,2022-05-22 08:25:47,2022-05-22 08:25:47


In [5]:
df = pd.merge(test_df, parsed_time_test_df, on=['date'], how="inner", indicator=True)
# df = df[df['_merge'] == 'left_only']
df = df[df['_merge'] == 'both']
df.head()

,id,date,date_orig_x,frame,date_orig_y,_merge
0,135,2022-05-22 11:51:06,2022-05-22 11:51:06,1159,2022-05-22 11:51:06,both
1,135,2022-05-22 11:51:06,2022-05-22 11:51:06,1160,2022-05-22 11:51:06,both
2,135,2022-05-22 11:51:06,2022-05-22 11:51:06,1161,2022-05-22 11:51:06,both
3,135,2022-05-22 11:51:06,2022-05-22 11:51:06,1162,2022-05-22 11:51:06,both
4,135,2022-05-22 11:51:06,2022-05-22 11:51:06,1163,2022-05-22 11:51:06,both


In [6]:
from catboost import CatBoostClassifier

door_model = CatBoostClassifier()
door_model.load_model("door_predict.cbm")

In [7]:
people_detect_model = torch.hub.load('ultralytics/yolov5','custom', 'yolov5x6') #'ultralytics/yolov5', 'custom', 'yolov5s-cls.pt'
people_detect_model.classes = [0]

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-8-10 Python-3.8.10 torch-1.13.0.dev20220701+cu116 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 


In [8]:
people_detect_model.cuda()
people_detect_model.eval()

AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 80, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1))
      

In [9]:
test_df.shape

(1121, 3)

In [10]:
from torch import nn
import torch.nn.functional as F
import timm # PyTorch Image Models

model = timm.create_model('tf_efficientnet_b4_ns',pretrained=True) #load pretrained model
for param in model.parameters():
    param.requires_grad=False

#orginally, it was:
#(classifier): Linear(in_features=1792, out_features=1000, bias=True)


#we are updating it as a 2-class classifier:
model.classifier = nn.Sequential(
    nn.Linear(in_features=1792, out_features=625), #1792 is the orginal in_features
    nn.ReLU(), #ReLu to be the activation function
    nn.Dropout(p=0.3),
    nn.Linear(in_features=625, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=10), 
)
model.to(device)
model.load_state_dict(torch.load('EmplDetectModel.pt'))
model.eval()

/usr/local/lib/python3.8/dist-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b4_ns-d6313a46.pth)


EfficientNet(
  (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          24, ep

In [11]:
from torchvision import transforms as T

train_transform = T.Compose([
     T.Resize(size=(224,224)), # Resizing the image to be 224 by 224
     T.ToTensor(), #converting the dimension from (height,weight,channel) to (channel,height,weight) convention of PyTorch
     T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) # Normalize by 3 means 3 StD's of the image net, 3 channels

])

In [12]:
employee_class_name = ['0','1','10','11','2','3','4','5','6','8']

In [13]:
test_df.head()

,id,date,date_orig
0,135,2022-05-22 11:51:06,2022-05-22 11:51:06
1,136,2022-05-22 11:51:25,2022-05-22 11:51:25
2,137,2022-05-22 12:04:27,2022-05-22 12:04:27
3,138,2022-05-22 12:07:56,2022-05-22 12:07:56
4,139,2022-05-22 12:08:10,2022-05-22 12:08:10


In [14]:
import cv2
from PIL import Image

test_path="/home/hacaton_irkutsk/test"
door_coord_items=[]
for ind in test_df.index:
    print(ind)
    row_id=int(test_df['id'][ind])
    row_frames_df=df[df['id']==row_id]
    
    uniq_frames=set()
    for fr_ind in row_frames_df.index:
        frame=int(row_frames_df['frame'][fr_ind])
        uniq_frames.add(frame)
        
    if len(uniq_frames)==0:
        continue
        
    min_frame=min(uniq_frames)
    max_frame=max(uniq_frames)
    ct=parsed_time_test_df['date_orig'][min_frame]
    
    for i in range(min_frame-10,max_frame+10):
        cframe=int(parsed_time_test_df['frame'][i])
        dt=parsed_time_test_df['date_orig'][i]
        if ct<dt:
            delta=dt-ct
        else:
            delta=ct-dt
        if delta.total_seconds()<4:
            uniq_frames.add(cframe)
        
    front_door_borders=[300,100,1050,1350]
    back_door_borders=[100,1000,400,1400]
    crop_boxes=[front_door_borders,back_door_borders]
    
    
    print('uniq_frames=',uniq_frames)
    for i in uniq_frames:
        img = cv2.imread(f"{test_path}/out{int(i)}.png", cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        newsize = (1920,1080)
        resized = cv2.resize(img, newsize, interpolation = cv2.INTER_AREA)
        
        for crop_box in crop_boxes:
            # front doors
            crop = resized[crop_box[0]:crop_box[2],crop_box[1]:crop_box[3]]

            results = people_detect_model(crop)
            if results.xyxy[0].shape == torch.Size([1, 6]):
#                 print('detect')
                find_region=results.xyxy[0][0].cpu().numpy()
                bbox=np.array(find_region[:4])
                bbox = bbox.astype(int)
                bbox[1]+=crop_box[0]
                bbox[3]+=crop_box[0]
                bbox[0]+=crop_box[1]
                bbox[2]+=crop_box[1]
                
                employee=0
                
                delta_y=0
                delta_x=0
                delta=(bbox[3]-bbox[1])-(bbox[2]-bbox[0])
                if delta>0:
                    delta_x=int(delta/2)
                else:
                    delta_y=int(delta/2)

                crop=resized[bbox[1]-delta_y:bbox[3]+delta_y, bbox[0]-delta_x:bbox[2]+delta_x]
#                 print(crop.shape)
                if crop.shape[0]!=0 and crop.shape[1]!=0:
                    im = Image.fromarray(crop)
                    resized_crop = train_transform(im)

    #                 print('resized_crop.shape=',resized_crop.shape)
                    ps = model(resized_crop.to(device).unsqueeze(0))
                    ps = F.softmax(ps,dim = 1)
                    ps = ps.cpu().data.numpy().squeeze()
                    employee_index=np.argmax(ps, axis=0)
                    employee=employee_class_name[employee_index]
                
                door_coord_items.append({
                    'row_id': row_id,
                    'frame_id':i,
                    'center_frame': min_frame,
                    'employee':employee,
                    'xmin':bbox[0],
                    'ymin':bbox[1],
                    'xmax':bbox[2],
                    'ymax':bbox[3],
                })
    


0
uniq_frames= {1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1150, 1151}
1
uniq_frames= {1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268}
2
uniq_frames= {1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 1400, 1401, 1402}
3
uniq_frames= {1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455}
4
uniq_frames= {1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525}
5
uniq_frames= {1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1532, 1533, 1534, 1535}
6
uniq_frames= {1664, 1665, 1666, 1667, 

53
uniq_frames= {3520, 3521, 3522, 3523, 3524, 3525, 3526, 3527, 3528, 3529, 3530, 3531, 3532, 3533, 3534, 3535, 3536, 3537, 3538, 3539, 3540, 3541}
54
uniq_frames= {3584, 3585, 3586, 3587, 3588, 3589, 3590, 3591, 3592, 3593, 3594, 3595, 3596, 3597, 3598, 3599, 3600, 3601, 3602, 3603, 3604, 3581, 3582, 3583}
55
uniq_frames= {3633, 3634, 3635, 3636, 3637, 3638, 3639, 3640, 3641, 3642, 3643, 3644, 3645, 3646, 3647, 3648, 3649, 3650, 3651, 3652, 3653, 3654, 3655, 3656}
56
uniq_frames= {3712, 3713, 3714, 3715, 3716, 3717, 3718, 3719, 3720, 3721, 3722, 3723, 3724, 3725, 3726, 3727, 3728, 3708, 3709, 3710, 3711}
57
uniq_frames= {3712, 3713, 3714, 3715, 3716, 3717, 3718, 3719, 3720, 3721, 3722, 3723, 3724, 3725, 3726, 3727, 3728, 3729, 3730, 3731, 3732, 3733, 3710, 3711}
58
uniq_frames= {3780, 3781, 3782, 3783, 3784, 3785, 3786, 3787, 3788, 3789, 3790, 3791, 3792, 3793, 3794, 3795, 3796, 3797, 3798, 3799, 3800, 3801, 3802, 3803}
59
uniq_frames= {3790, 3791, 3792, 3793, 3794, 3795, 3796, 3797,

105
uniq_frames= {5440, 5441, 5442, 5443, 5444, 5445, 5446, 5447, 5448, 5449, 5450, 5451, 5452, 5453, 5454, 5455, 5456, 5457, 5458, 5459, 5460, 5461, 5462, 5463}
106
uniq_frames= {5504, 5505, 5506, 5507, 5508, 5509, 5510, 5511, 5512, 5494, 5495, 5496, 5497, 5498, 5499, 5500, 5501, 5502, 5503}
107
uniq_frames= {5509, 5510, 5511, 5512, 5513, 5514, 5515, 5516, 5517, 5518, 5519, 5520, 5521, 5522, 5523, 5524, 5525, 5526, 5527, 5528, 5529, 5530, 5531, 5532}
108
109
110
uniq_frames= {5662, 5663, 5664, 5665, 5666, 5667, 5668, 5669, 5670, 5671, 5672, 5673, 5674, 5675, 5676, 5677, 5678, 5679, 5680, 5681, 5682, 5683}
111
uniq_frames= {5795, 5796, 5797, 5798, 5799, 5800, 5801, 5802, 5803, 5804, 5805, 5806, 5807, 5808, 5809, 5810, 5811, 5812, 5813, 5814, 5815, 5816, 5817, 5818}
112
uniq_frames= {5889, 5890, 5891, 5892, 5893, 5894, 5895, 5896, 5897, 5898, 5899, 5900, 5901, 5902, 5903, 5904, 5905, 5906, 5907, 5908, 5909, 5910, 5911, 5912}
113
uniq_frames= {5974, 5975, 5976, 5977, 5978, 5979, 5980, 59

160
uniq_frames= {8102, 8103, 8104, 8105, 8106, 8107, 8108, 8109, 8110, 8111, 8112, 8113, 8114, 8115, 8116, 8117, 8118, 8119, 8120, 8121, 8122, 8123, 8124, 8125}
161
uniq_frames= {8122, 8123, 8124, 8125, 8126, 8127, 8128, 8129, 8130, 8131, 8132, 8133, 8134, 8135, 8136, 8137, 8138, 8139, 8140, 8141, 8142, 8143, 8144, 8145}
162
uniq_frames= {8205, 8206, 8207, 8208, 8209, 8210, 8211, 8212, 8213, 8214, 8215, 8216, 8217, 8218}
163
uniq_frames= {8221, 8222, 8223, 8224, 8225, 8226, 8227, 8228, 8229, 8230, 8231, 8232, 8233, 8234, 8235, 8236, 8237, 8238, 8239, 8240, 8241, 8242, 8243, 8244}
164
uniq_frames= {8231, 8232, 8233, 8234, 8235, 8236, 8237, 8238, 8239, 8240, 8241, 8242, 8243, 8244, 8245, 8246, 8247, 8248, 8249, 8250, 8251, 8252, 8253, 8254}
165
uniq_frames= {8281, 8282, 8283, 8284, 8285, 8286, 8287, 8288, 8289, 8290, 8291, 8292, 8293, 8294, 8295, 8296, 8297, 8298, 8299, 8300, 8301, 8302, 8303, 8304}
166
uniq_frames= {8322, 8323, 8324, 8325, 8326, 8327, 8328, 8329, 8330, 8331, 8332, 8333

212
uniq_frames= {10498, 10499, 10500, 10501, 10502, 10503, 10504, 10505, 10506, 10507, 10508, 10509, 10510, 10511, 10512, 10513, 10514, 10515, 10516, 10517, 10518, 10519, 10520, 10521}
213
uniq_frames= {10553, 10554, 10555, 10556, 10557, 10558, 10559, 10560, 10561, 10562, 10563, 10564, 10565, 10566, 10567, 10568, 10569, 10570, 10571, 10572, 10573, 10574}
214
uniq_frames= {10581, 10582, 10583, 10584, 10585, 10586, 10587, 10588, 10589, 10590, 10591, 10592, 10593, 10594, 10595, 10596, 10597, 10598, 10599, 10600, 10601, 10602, 10603, 10604}
215
uniq_frames= {10651, 10652, 10653, 10654, 10655, 10656, 10657, 10658, 10659, 10660, 10661, 10662, 10663, 10664, 10665, 10666, 10667, 10668, 10669, 10670, 10671, 10672, 10673, 10674}
216
uniq_frames= {10696, 10697, 10698, 10699, 10700, 10701, 10702, 10703, 10704, 10705, 10706, 10707, 10708, 10709, 10710, 10711, 10712, 10713, 10714, 10715, 10716, 10717, 10718, 10719}
217
uniq_frames= {10769, 10770, 10771, 10772, 10773, 10774, 10775, 10776, 10777, 107

...

893
uniq_frames= {49610, 49611, 49612, 49613, 49614, 49615, 49616, 49617, 49618, 49619, 49620, 49621, 49622, 49623, 49624, 49625, 49626, 49627, 49628, 49629, 49630, 49631, 49632, 49633}
894
uniq_frames= {49675, 49676, 49677, 49678, 49679, 49680, 49681, 49682, 49683, 49684, 49685, 49686, 49687, 49688, 49689, 49690, 49691, 49692, 49693, 49694, 49695, 49696, 49697, 49698}
895
uniq_frames= {49722, 49723, 49724, 49725, 49726, 49727, 49728, 49729, 49730, 49731, 49732, 49733, 49734, 49735, 49736, 49737, 49738, 49739, 49740, 49741}
896
uniq_frames= {49792, 49793, 49794, 49795, 49796, 49797, 49798, 49799, 49800, 49801, 49802, 49803, 49804, 49805, 49806, 49807, 49808, 49809, 49810, 49811, 49788, 49789, 49790, 49791}
897
uniq_frames= {49837, 49838, 49839, 49840, 49841, 49842, 49843, 49844, 49845, 49846, 49847, 49848, 49849, 49850, 49851, 49852, 49853, 49854, 49855, 49856}
898
uniq_frames= {49863, 49864, 49865, 49866, 49867, 49868, 49869, 49870, 49871, 49872, 49873, 49874, 49875, 49876, 49877, 498

939
940
uniq_frames= {51842, 51843, 51844, 51845, 51846, 51847, 51848, 51849, 51850, 51851, 51852, 51853, 51854, 51855, 51856, 51857, 51858, 51859, 51860, 51861, 51862, 51863, 51864, 51865}
941
uniq_frames= {51883, 51884, 51885, 51886, 51887, 51888, 51889, 51890, 51891, 51892, 51893, 51894, 51895, 51896}
942
uniq_frames= {52011, 52012, 52013, 52014, 52015, 52016, 52017, 52018, 52019, 52020, 52021, 52022, 52023, 52024, 52025, 52026, 52027, 52028, 52029, 52030, 52031, 52032, 52033, 52034}
943
uniq_frames= {52228, 52229, 52230, 52231, 52232, 52233, 52234, 52235, 52236, 52237, 52238, 52239, 52240, 52241, 52242, 52243, 52244, 52245, 52246, 52247, 52248, 52249, 52250, 52251}
944
uniq_frames= {52453, 52454, 52455, 52456, 52457, 52458, 52459, 52460, 52461, 52462, 52463, 52464, 52465, 52466, 52467, 52468, 52469, 52470, 52471, 52472, 52473, 52474, 52475, 52476}
945
uniq_frames= {52498, 52499, 52500, 52501, 52502, 52503, 52504, 52505, 52506, 52507, 52508, 52509, 52510, 52511, 52512, 52513, 52514,

988
uniq_frames= {55266, 55267, 55268, 55269, 55270, 55271, 55272, 55273, 55274, 55275, 55276, 55277, 55278, 55279, 55280, 55281, 55282, 55283, 55284, 55285, 55286, 55287, 55288, 55289}
989
uniq_frames= {55322, 55323, 55324, 55325, 55326, 55327, 55328, 55329, 55330, 55331, 55332, 55333, 55334, 55335, 55336, 55337, 55338, 55339, 55340, 55341, 55342, 55343, 55344, 55345}
990
uniq_frames= {55357, 55358, 55359, 55360, 55361, 55362, 55363, 55364, 55365, 55366, 55367, 55368, 55369, 55370, 55371, 55372, 55373, 55374, 55375, 55376, 55377, 55378, 55379, 55380}
991
uniq_frames= {55424, 55425, 55402, 55403, 55404, 55405, 55406, 55407, 55408, 55409, 55410, 55411, 55412, 55413, 55414, 55415, 55416, 55417, 55418, 55419, 55420, 55421, 55422, 55423}
992
uniq_frames= {55437, 55438, 55439, 55440, 55441, 55442, 55443, 55444, 55445, 55446, 55447, 55448, 55449, 55450, 55451, 55452, 55453, 55454, 55455, 55456, 55457, 55458, 55459, 55460}
993
uniq_frames= {55505, 55506, 55507, 55508, 55509, 55510, 55511, 555

1033
uniq_frames= {58331, 58332, 58333, 58334, 58335, 58336, 58337, 58338, 58339, 58340, 58341, 58342, 58343, 58344, 58345, 58346, 58347, 58348, 58349, 58350, 58351, 58352, 58353, 58354}
1034
uniq_frames= {58447, 58448, 58449, 58450, 58451, 58452, 58453, 58454, 58455, 58456, 58457, 58458, 58459, 58460, 58461, 58462, 58463, 58464, 58465, 58466, 58467, 58468, 58469, 58470}
1035
uniq_frames= {58631, 58632, 58633, 58634, 58635, 58636, 58637, 58638, 58639, 58640, 58641, 58642, 58643, 58644, 58645, 58646, 58647, 58648, 58649, 58650, 58651, 58652, 58653}
1036
uniq_frames= {58645, 58646, 58647, 58648, 58649, 58650, 58651, 58652, 58653, 58654, 58655, 58656, 58657, 58658, 58659, 58660, 58661, 58662, 58663, 58664, 58665, 58666, 58667}
1037
uniq_frames= {58687, 58688, 58689, 58690, 58691, 58692, 58693, 58694, 58695, 58696, 58697, 58698, 58699, 58700, 58701, 58702, 58703, 58704, 58705, 58706, 58707, 58708}
1038
uniq_frames= {58785, 58786, 58787, 58788, 58789, 58790, 58791, 58792, 58793, 58794, 5879

1078
uniq_frames= {60494, 60495, 60496, 60497, 60498, 60499, 60500, 60501, 60502, 60503, 60504, 60505, 60506, 60507, 60508, 60509, 60510, 60511, 60512, 60513, 60514, 60515, 60516, 60517}
1079
uniq_frames= {60499, 60500, 60501, 60502, 60503, 60504, 60505, 60506, 60507, 60508, 60509, 60510, 60511, 60512, 60513, 60514, 60515, 60516, 60517, 60518, 60519, 60520, 60521, 60522}
1080
uniq_frames= {60504, 60505, 60506, 60507, 60508, 60509, 60510, 60511, 60512, 60513, 60514, 60515, 60516, 60517, 60518, 60519, 60520, 60521, 60522, 60523, 60524, 60525, 60526, 60527}
1081
uniq_frames= {60514, 60515, 60516, 60517, 60518, 60519, 60520, 60521, 60522, 60523, 60524, 60525, 60526, 60527, 60528, 60529, 60530, 60531, 60532, 60533, 60534, 60535, 60536, 60537}
1082
uniq_frames= {60574, 60575, 60576, 60577, 60578, 60579, 60580, 60581, 60582, 60583, 60584, 60585, 60586, 60587, 60588, 60589, 60590, 60591, 60592, 60593, 60594, 60595, 60596, 60597}
1083
uniq_frames= {60709, 60710, 60711, 60712, 60713, 60714, 6071

In [15]:
len(door_coord_items)

19631

In [16]:
door_coord_df = pd.DataFrame(door_coord_items)
door_coord_df.head(20)

,row_id,frame_id,center_frame,employee,xmin,ymin,xmax,ymax
0,135,1163,1159,0,1296,268,1316,342
1,135,1164,1159,3,1277,268,1309,336
2,135,1165,1159,0,1273,266,1299,336
3,135,1166,1159,0,1264,265,1289,335
4,135,1167,1159,3,1254,265,1287,336
5,135,1168,1159,3,1250,264,1273,335
6,135,1169,1159,3,1238,263,1267,334
7,135,1170,1159,0,1229,264,1260,334
8,135,1171,1159,3,1230,263,1255,335
9,135,1172,1159,0,1226,300,1242,333


In [17]:
door_coord_df.shape

(19631, 8)

In [18]:
pred = door_model.predict(door_coord_df[['xmin','ymin','xmax','ymax']])
door_coord_df['pred']=pred
door_coord_df.head()

,row_id,frame_id,center_frame,employee,xmin,ymin,xmax,ymax,pred
0,135,1163,1159,0,1296,268,1316,342,5
1,135,1164,1159,3,1277,268,1309,336,5
2,135,1165,1159,0,1273,266,1299,336,5
3,135,1166,1159,0,1264,265,1289,335,4
4,135,1167,1159,3,1254,265,1287,336,5


In [19]:
door_coord_df[door_coord_df['row_id']==245].head(40)

,row_id,frame_id,center_frame,employee,xmin,ymin,xmax,ymax,pred
1433,245,5672,5669,3,1288,262,1322,343,5
1434,245,5673,5669,10,1288,259,1316,342,5
1435,245,5674,5669,0,1276,259,1315,338,5
1436,245,5675,5669,6,1266,300,1299,342,2
1437,245,5675,5669,6,1269,260,1303,342,5
1438,245,5676,5669,2,1268,300,1295,340,2
1439,245,5676,5669,3,1267,259,1298,341,5
1440,245,5677,5669,4,1266,300,1294,344,2
1441,245,5677,5669,3,1263,260,1298,344,5
1442,245,5678,5669,6,1264,300,1289,346,2


In [20]:

door_coord_df[['employee']] = door_coord_df[['employee']].astype(int)

In [21]:
result_actions={}
result_employee={}
for ind in test_df.index:
    row_id=int(test_df['id'][ind])
    actions_df=door_coord_df[door_coord_df['row_id']==row_id]
    actions_df = actions_df.sort_values(by ='frame_id' )
    if actions_df.shape[0]==0:
        continue
    
    frame_preds={}
    for i in actions_df.index:
        center_frame=actions_df['center_frame'][i] 
        print(center_frame)
        frame_id=actions_df['frame_id'][i]
        
        door=actions_df['pred'][i]
        frame_preds[frame_id]=door
#         print(center_frame,frame_id,door)
    print(center_frame)

    prev_count={}
    next_count={}
    row_frames=frame_preds.keys()
    sorted(row_frames, reverse=True)
    for f in row_frames:
        if f>center_frame:
            continue
            
        if frame_preds[f] not in prev_count:
            prev_count[frame_preds[f]]=0
        prev_count[frame_preds[f]]+=1
    sorted(row_frames, reverse=False)
    for f in row_frames:
        if f<=center_frame:
            continue
            
        if frame_preds[f] not in next_count:
            next_count[frame_preds[f]]=0
        next_count[frame_preds[f]]+=1
        
    prev_max_val=-1
    prev_max_val_door=-1
    for k in prev_count:
        if prev_count[k]>prev_max_val:
            prev_max_val=prev_count[k]
            prev_max_val_door=k
            
    next_max_val=-1
    next_max_val_door=-1
    for k in next_count:
        if next_count[k]>next_max_val:
            next_max_val=next_count[k]
            next_max_val_door=k
    
    if prev_max_val>=next_max_val:
        empl_df=actions_df[actions_df['frame_id']<=center_frame].tail(5)
        result_actions[row_id]=prev_max_val_door
    else:
        empl_df=actions_df[actions_df['frame_id']>=center_frame].head(5)
        result_actions[row_id]=next_max_val_door+10
        
    values, counts = np.unique(empl_df['employee'], return_counts=True)
    freq_ind = np.argmax(counts)

    result_employee[row_id]=values[freq_ind]
    
        
        
len(result_actions)
    
            
            

1159
1159
1159
1159
1159
1159
1159
1159
1159
1159
1159
1159
1159
1254
1254
1254
1254
1254
1254
1254
1254
1254
1254
1254
1254
1254
1254
1388
1388
1388
1388
1388
1388
1388
1388
1388
1388
1388
1388
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1441
1541
1541
1541
1541
1541
1541
1541
1541
1671
1671
1671
1671
1671
1781
1781
1781
1781
1781
1781
1781
1781
1781
1781
1781
1781
1781
1781
1781
1781
1841
1841
1841
1841
1841
1841
1841
1841
1841
1841
1866
1866
1866
1866
1866
1866
1866
1866
1866
1866
1866
1866
2096
2096
2096
2096
2096
2096
2096
2096
2096
2129
2129
2129
2129
2129
2129
2154
2154
2154
2154
2191
2191
2191
2191
2191
2191
2191
2191
2191
2201
2201
2201
2201
2201
2201
2201
2201
2201
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2236
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241
2241


8061
8061
8061
8061
8061
8061
8061
8061
8061
8061
8061
8061
8061
8061
8061
8061
8061
8061
8111
8111
8111
8111
8111
8111
8111
8111
8111
8111
8111
8111
8111
8131
8131
8131
8131
8131
8131
8131
8131
8131
8131
8131
8131
8131
8131
8131
8131
8131
8214
8214
8214
8214
8214
8214
8214
8214
8214
8214
8214
8214
8214
8214
8214
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8230
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8240
8290
8290
8290
8290
8290
8290
8290
8290
8290
8290
8290
8290
8290
8290
8290
8290
8290
8290
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8331
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8373
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8453
8519
8519
8519
8519
8519
8519
8519
8519
8519
8519
8519
8519
8594
8594


14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14509
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14524
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14594
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14614
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14665
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14705
14767
14767
14767
14767
14767
14767
14767
14767
14767
14767
14767
14767
14767
14767
14767
14767
14767
1476

22887
22887
22887
22887
22887
22887
22887
22887
22887
22887
22887
22887
22887
22887
22887
23237
23237
23237
23237
23237
23237
23237
23237
23237
23237
23237
23237
23237
23237
23237
23237
23237
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23265
23305
23305
23305
23305
23305
23305
23305
23305
23305
23305
23305
23305
23305
23305
23305
23305
23305
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23351
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23386
23436
23436
23436
23436
23436
23436
23436
23436
23436
23436
23436
23436
23436
23436
23436
23436
23436
23563
23563
23563
23563
23563
23563
23563
23618
23618
23708
23708
23708
23708
23708
23708
23708
23708
23708
23708
23708
23708
23708
23708
23708
23708
23708
23708
23748
23748
23748
23748
23748
23748
23748
2374

31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31550
31668
31668
31668
31668
31668
31668
31668
31668
31668
31708
31708
31708
31708
31708
31708
31708
31708
31708
31713
31713
31713
31713
31713
31713
31713
31713
31713
31766
31766
31766
31766
31766
31766
31766
31766
31766
31766
31766
31766
31766
31791
31791
31791
31791
31791
31791
31791
31791
31791
31791
31791
31791
31791
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31837
31892
31892
31892
31892
31892
31892
31892
31892
31892
32012
32012
32012
32012
32012
32012
32012
32012
32012
32012
32012
32012
32012
32012
32012
32012
32012
32057
32057
32057
32057
32057
32057
32057
32057
3205

39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39448
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39468
39519
39519
39519
39519
39519
39519
39519
39519
39519
39519
39519
39519
39519
39519
39519
39519
39519
39519
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39529
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39581
39596
39596
39596
39596
39596
39596
39596
39596
39596
39596
39596
39596
39596
39596
39596
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39649
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
39709
3970

42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42527
42662
42662
42662
42662
42662
42662
42662
42662
42662
42662
42662
42662
42737
42737
42737
42737
42737
42737
42737
42737
42737
42737
42737
42737
42737
42737
42737
42737
42737
42737
42788
42788
42788
42788
42788
42788
42788
42788
42788
42788
42788
42788
42788
42788
42788
42788
42788
42808
42808
42808
42808
42808
42808
42808
42808
42808
42808
42808
42808
42808
42808
42808
42808
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42861
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42921
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42971
42981
42981
42981
42981
42981
42981
42981
42981
42981
42981
42981
42981
42981
42981
42981
4298

45983
45983
45983
45983
45983
45983
45983
45983
45983
45983
45983
45983
45983
46043
46043
46043
46043
46043
46043
46043
46043
46043
46043
46043
46043
46043
46073
46073
46073
46073
46073
46073
46073
46073
46073
46073
46073
46073
46073
46073
46073
46073
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46078
46103
46103
46103
46103
46103
46103
46103
46103
46103
46103
46103
46103
46103
46103
46103
46103
46103
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46123
46133
46133
46133
46133
46133
46133
46133
46133
46133
46133
46133
46133
46133
46168
46168
46168
46168
46168
46168
46168
46168
46168
46168
46168
46168
46168
46168
46168
46168
46168
46238
46238
46238
46238
46238
46238
46238
4623

52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52900
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52935
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
52972
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53087
53134
53134
53134
53134
53134
53134
53134
53287
53287
53287
53287
53287
53287
53287
53287
53287
53287
53287
53287
53287
53287
53287
53387
53387
53387
53387
53387
53387
53387
53387
53387
53387
53472
53472
53472
53472
53472
53472
53472
53472
53472
53472
53472
53472
53472
53472
53472
53472
53472
53482
53482
53482
53482
53482
53482
53482
53482
53482
53482
53536
53536
53536
53536
53536
53536
53536
53536
53536
53536
53536
53536
53536
53536
53536
53536
5353

1087

In [22]:
result_actions[245]

15

In [23]:
result_employee[245]

6

In [24]:
with open("irkutsk/sample_solution.csv","rt") as f:
    lines = f.readlines()

In [25]:
with open("submit_5.csv","wt") as f:
    f.write("id,employee,action\n")
    for line in lines[1:]:
        id,empl,act=line.strip().split(',')
        id=int(id)
        if id in result_actions and result_actions[id]>0:
            act=result_actions[id]
        if id in result_employee:
            empl=int(result_employee[id])
            if empl==7:
                empl=0
        f.write(f"{id},{empl},{act}\n")


In [26]:
print(id,empl,act)

1255 0 0
